In [2]:
#필요한 모드 임포트
import os.path
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
from PIL import Image

from pathlib import Path
from tqdm import tqdm
from time import perf_counter

from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score
from IPython.display import Markdown, display
from keras.preprocessing.image import ImageDataGenerator

# 저장된 모델 로드
model = tf.keras.models.load_model('test_model.h5')

In [3]:
#이미지 파일경로 저장 및 데이터 라벨링
dir_ = Path('C:/Users/alstn/Desktop/153.반려동물 안구질환 데이터/01.데이터/1.Training/원천데이터/TS1/개/안구/일반')
test_dir_ = Path('C:/Users/alstn/Desktop/153.반려동물 안구질환 데이터/01.데이터/2.Validation/원천데이터/VS/개/안구/일반/')

import glob
types = ('**/*.jpg', '**/*.png','**/*.jpeg') # the tuple of file types
files_grabbed = []
test_files_grabbed = []
for files in types:
    files_grabbed.extend(dir_.glob(files))
    test_files_grabbed.extend(test_dir_.glob(files))
    
def proc_img(filepath):
    """
   		이미지데이터의 경로와 label데이터로 데이터프레임 만들기 
    """

    labels = []
    for i in range(len(filepath)):
        if(str(filepath[i]).split("\\")[-2] == '무'):
            labels.append(str(filepath[i]).split("\\")[-2])
        else:
            labels.append(str(filepath[i]).split("\\")[-3])

    filepath = pd.Series(filepath, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')
             
    # 경로와 라벨 concatenate
    df = pd.concat([filepath, labels], axis=1)

    # index 재설정
    df = df.sample(frac=1,random_state=0).reset_index(drop = True)
    
    return df

df = proc_img(files_grabbed)
test_df = proc_img(test_files_grabbed)
df.head(5)

,Filepath,Label
0,C:\Users\alstn\Desktop\153.반려동물 안구질환 데이터\01.데이...,백내장
1,C:\Users\alstn\Desktop\153.반려동물 안구질환 데이터\01.데이...,백내장
2,C:\Users\alstn\Desktop\153.반려동물 안구질환 데이터\01.데이...,무
3,C:\Users\alstn\Desktop\153.반려동물 안구질환 데이터\01.데이...,결막염
4,C:\Users\alstn\Desktop\153.반려동물 안구질환 데이터\01.데이...,무


In [5]:
def create_gen():
    # 생성기 및 데이터 증강으로 이미지 로드
    train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
        validation_split=0.2
    )

    test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
    )

    train_images = train_generator.flow_from_dataframe(
        dataframe=df,
        x_col='Filepath', # 파일위치 열이름
        y_col='Label', # 클래스 열이름
        target_size=(224, 224), # 이미지 사이즈
        color_mode='rgb', # 이미지 채널수
        class_mode='categorical', # Y값(Label값)
        batch_size=16,
        shuffle=True, # 데이터를 섞을지 여부
        seed=0,
        subset='training', # train 인지 val인지 설정
        rotation_range=30, # 회전제한 각도 30도
        zoom_range=0.15, # 확대 축소 15%
        width_shift_range=0.2, # 좌우이동 20%
        height_shift_range=0.2, # 상하이동 20%
        shear_range=0.15, # 반시계방햐의 각도
        horizontal_flip=True, # 좌우 반전 True
        fill_mode="nearest"
        # 이미지 변경시 보완 방법 (constant, nearest, reflect, wrap) 4개 존재
    )

    val_images = train_generator.flow_from_dataframe(
        dataframe=df,
        x_col='Filepath',
        y_col='Label',
        target_size=(224, 224),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=16,
        shuffle=True,
        seed=0,
        subset='validation',
        rotation_range=30,
        zoom_range=0.15,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        horizontal_flip=True,
        fill_mode="nearest"
    )

    test_images = test_generator.flow_from_dataframe(
        dataframe=test_df,
        x_col='Filepath',
        y_col='Label',
        target_size=(224, 224),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=16,
        shuffle=False
    )
    
    return train_generator,test_generator,train_images,val_images,test_images

In [8]:
train_generator,test_generator,train_images,val_images,test_images=create_gen()

Found 141452 validated image filenames belonging to 11 classes.
Found 35363 validated image filenames belonging to 11 classes.
Found 22058 validated image filenames belonging to 11 classes.


In [24]:
pred = model.predict(test_images)
pred = np.argmax(pred, axis=1)

labels = train_images.class_indices
labels = {v: k for k, v in labels.items()}
pred = [labels[k] for k in pred]

def get_accuracy(y_true, y_pred):
    acc = accuracy_score(y_true, y_pred)
    return f"Accuracy on the test set: {acc * 100:.2f}%"

y_test = list(test_df.Label)
acc = get_accuracy(y_test, pred)
print(acc)

1379/1379 [==============================] - 424s 308ms/step
Accuracy on the test set: 62.36%


In [35]:
pred = model.predict(test_images)  # 테스트 이미지에 대한 예측 값을 가져옵니다.
pred_top3 = np.argsort(pred, axis=1)[:, -3:]  # 상위 3개 예측 값의 인덱스를 가져옵니다.
y_test = list(test_df.Label)  # 테스트 데이터의 실제 레이블을 가져옵니다.

count = 0  # 정확한 예측의 개수를 저장할 변수를 초기화합니다.
arr_count = 0  # 테스트 데이터의 인덱스를 추적하기 위한 변수를 초기화합니다.

top3_values = []  # 상위 3개 예측 값들을 저장할 리스트를 생성합니다.

# 각 행에 대해 예측된 상위 3개 값의 인덱스를 해당하는 레이블로 변환하여 리스트에 추가합니다.
for indices in pred_top3:
    top3_values.append([labels[idx] for idx in indices])

# 각 예측 값에 대해 실제 값과 비교하여 정확한 예측인지 확인합니다.
for values in top3_values:
    for value in values:
        if value == y_test[arr_count]:
            count += 1  # 정확한 예측의 개수를 증가시킵니다.
            break
    arr_count += 1  # 다음 테스트 데이터의 인덱스로 이동합니다.

accuracy = count / len(y_test)  # 정확도를 계산합니다.
print(f"Accuracy on the test set: {accuracy * 100:.2f}%")  # 정확도를 백분율로 출력합니다.


1379/1379 [==============================] - 429s 311ms/step
Accuracy on the test set: 84.35%
